1. Carga e inspeccion de datos

In [0]:
# Librerias
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import mlflow
import mlflow.sklearn

In [0]:
# Cargar el conjunto de datos
file_path = '../EDA/ObesityDataSet.csv'
df = pd.read_csv(file_path)

# Previamente ya se ha hecho un analisis en el EDA (EDA/EDA - Risk Obesity)

# Información general y tipos de datos
print("Información General")
df.info()

# Primeras filas
print("Primeras Filas")
print(df.head())

# Conteo de valores nulos
print("Conteo de Valores Nulos")
print(df.isnull().sum())

# Distribución de la variable objetivo
print("Distribución de la Variable Objetivo (NObeyesdad)")
print(df['NObeyesdad'].value_counts())

# Valores únicos en columnas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns
print("Valores Únicos en Columnas Categóricas")
for col in categorical_cols:
    print(f"Valores en '{col}': {df[col].unique()}")

2. Definición de Features (X) y Target (y)

In [0]:
# X contiene todas las columnas EXCEPTO la objetivo
X = df.drop('NObeyesdad', axis=1)

# y contiene ÚNICAMENTE la columna objetivo
y = df['NObeyesdad']

print("Forma de X (features):", X.shape)
print("Forma de y (target):", y.shape)

3. Codificación de la Variable Objetivo (y)

In [0]:
# Inicializar el codificador
le = LabelEncoder()

# Ajustar y transformar y
y_encoded = le.fit_transform(y)

# Guardar los nombres de las clases para los reportes
class_names = le.classes_

print("Etiquetas originales:", y.unique()[:5])
print("Etiquetas codificadas:", y_encoded[:5])
print("Nombres de clases guardados:", class_names)

4. División de Datos (Entrenamiento y Prueba)

In [0]:
# Dividir 80% entrenamiento, 20% prueba
# Se usa stratify=y_encoded para asegurar que la distribución de clases sea la misma en ambos conjuntos 
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"Registros de entrenamiento (X_train): {X_train.shape[0]}")
print(f"Registros de prueba (X_test): {X_test.shape[0]}")

In [0]:
# ======================================================
# Configuración de MLflow
# ======================================================
mlflow.set_tracking_uri("http://50.17.8.104:8050")
mlflow.set_experiment("ObesityRisk_Models")

5. Definición del Pipeline de Preprocesamiento

In [0]:
# Definir listas de columnas
# Columnas numéricas
numeric_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
# Columnas ordinales
ordinal_features = ['CAEC', 'CALC']
# Definimos el orden de las categorías para cada columna
caec_categories = ['no', 'Sometimes', 'Frequently', 'Always']
calc_categories = ['no', 'Sometimes', 'Frequently', 'Always']
# Columnas nominales y binarias
nominal_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']


# Crear los transformadores
# Transformador para numéricas: Escalar
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Transformador para ordinales: Codificar por orden
ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories=[caec_categories, calc_categories], 
                             handle_unknown='use_encoded_value', 
                             unknown_value=-1))
])

# Transformador para nominales: Crear dummies 
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])


# Unir todo en el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='passthrough'
)

6. Creación y Entrenamiento: Modelo 1 (Regresión Logística)

In [0]:
# ======================================================
# Modelo 1: Regresión Logística
# ======================================================
pipe_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='lbfgs', max_iter=1500, random_state=42))
])

print("Entrenando modelo de Regresión Logística")
with mlflow.start_run(run_name="Logistic_Regression"):
    pipe_lr.fit(X_train, y_train)
    y_pred_lr = pipe_lr.predict(X_test)

    # === Reporte en consola ===
    print("\nREPORTE DE REGRESIÓN LOGÍSTICA")
    print(classification_report(y_test, y_pred_lr, target_names=class_names))

    # === Registro en MLflow ===
    report_lr = classification_report(y_test, y_pred_lr, target_names=class_names, output_dict=True)
    
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_metric("accuracy", report_lr["accuracy"])
    for cls in class_names:
        if cls in report_lr:
            mlflow.log_metric(f"{cls}_precision", report_lr[cls]["precision"])
            mlflow.log_metric(f"{cls}_recall", report_lr[cls]["recall"])
            mlflow.log_metric(f"{cls}_f1", report_lr[cls]["f1-score"])
    mlflow.sklearn.log_model(pipe_lr, "model")

print("Entrenamiento completado para Regresión Logística")

7. Creación y Entrenamiento: Modelo 2 (Random Forest)

In [0]:
# ======================================================
# Modelo 2: Random Forest
# ======================================================
pipe_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])

print("Entrenando modelo de Random Forest")
with mlflow.start_run(run_name="Random_Forest"):
    pipe_rf.fit(X_train, y_train)
    y_pred_rf = pipe_rf.predict(X_test)

    # === Reporte en consola ===
    print("\nREPORTE DE RANDOM FOREST")
    print(classification_report(y_test, y_pred_rf, target_names=class_names))

    # === Registro en MLflow ===
    report_rf = classification_report(y_test, y_pred_rf, target_names=class_names, output_dict=True)
    
    mlflow.log_param("model", "RandomForestClassifier")
    mlflow.log_metric("accuracy", report_rf["accuracy"])
    for cls in class_names:
        if cls in report_rf:
            mlflow.log_metric(f"{cls}_precision", report_rf[cls]["precision"])
            mlflow.log_metric(f"{cls}_recall", report_rf[cls]["recall"])
            mlflow.log_metric(f"{cls}_f1", report_rf[cls]["f1-score"])
    mlflow.sklearn.log_model(pipe_rf, "model")

print("Entrenamiento completado para Random Forest")

8. Evaluación: Reportes de Clasificación

In [0]:
# Predicciones
y_pred_lr = pipe_lr.predict(X_test)
y_pred_rf = pipe_rf.predict(X_test)

# Reportes
print("REPORTE DE REGRESIÓN LOGÍSTICA")
print(classification_report(y_test, y_pred_lr, target_names=class_names))

print("\n")
print("REPORTE DE RANDOM FOREST")
print(classification_report(y_test, y_pred_rf, target_names=class_names))

9. Evaluación: Matrices de Confusión

In [0]:
# Crear una figura con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10))

# Matriz de Regresión Logística
ConfusionMatrixDisplay.from_estimator(
    pipe_lr, X_test, y_test,
    ax=ax1,
    cmap='Blues',
    xticks_rotation='vertical',
    display_labels=class_names
)
ax1.set_title('Matriz de Confusión: Regresión Logística', fontsize=16)

# Matriz de Random Forest
ConfusionMatrixDisplay.from_estimator(
    pipe_rf, X_test, y_test,
    ax=ax2,
    cmap='Greens',
    xticks_rotation='vertical',
    display_labels=class_names
)
ax2.set_title('Matriz de Confusión: Random Forest', fontsize=16)

plt.tight_layout()